# ACCESS-OM3 SST and SST difference from WOA23

Related issue: https://github.com/ACCESS-Community-Hub/access-om3-25km-paper-1/issues/9

In [1]:
#This cell must be in all notebooks!
#It allows us to run all the notebooks at once, this cell has a tag "parameters" which allows us to pass in 
# arguments externally using papermill (see mkfigs.sh for details)

### USER EDIT start
esm_file='/g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json'
dpi=300
### USER EDIT stop

import os
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.dpi']= dpi

plotfolder=f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-figs/"
os.makedirs(plotfolder, exist_ok=True)

 # a similar cell under this means it's being run in batch
print("ESM datastore path: ",esm_file)
print("Plot folder path: ",plotfolder)

ESM datastore path:  /g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json
Plot folder path:  /g/data/g40/jj8842/access-om3-paper-figs/


In [2]:
import xarray as xr
import cf_xarray
import intake
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from distributed import Client

In [3]:
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45799,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:45751,Total threads: 1
Dashboard: /proxy/38337/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:41853,


### Define plot function

In [4]:
blue_marble = plt.imread('/g/data/ik11/grids/BlueMarble.tiff')
blue_marble_extent = (-180, 180, -90, 90)

In [5]:
def plot(dat, title=None, **kwargs):
    if title is None:
        title = dat.attrs['long_name']
    fig = plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=-100))
    dat.plot.contourf(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cbar_kwargs={"label": "\n".join(wrap(f"{dat.attrs['long_name']} [{dat.attrs['units']}]", 45)),
                     "fraction": 0.03, "aspect": 15, "shrink": 0.7},
        **kwargs
    )
    
    # Add blue marble land:
    ax.imshow(
        blue_marble, extent=blue_marble_extent, transform=ccrs.PlateCarree(), origin="upper"
    )
    
    plt.title(title);

## Define 20° isotherm function

In [4]:
def interpolate_isotherm_depth(temp, target_temp=20, depth_name='z_t'):
    """ 
        Interpolates between the deepest point warmer than target_temp 
        and the next deepest point to return isotherm depth

        Parameters
        ----------
        temp : xr.DataArray
            N-dimensional ocean temperature, where depth is one dimension
            CMIP name would be thetao
        target_temp: scalar
            temperature of isotherm being calculated
        depth_name: str
            name of xarray dimension on temp containing depth

        Returns
        -------
        xr.DataArray
            isotherm depth, same shape as temp but with the depth coordinate collapsed

        Notes
        -----
        Anywhere the surface (first depth) value is NaN returns a NaN
        Anywhere that the entire water collumn is colder than target_temp returns a zero
        Anywhere that the entire water collumn is warmer than target_temp returns an 
            interpolation between the deepest layer and the next one down
        Poorly defined if the isotherm hits the bottom of the grid, but I can't see that
            happening in a reasonable situation, will throw errors
    """    
    assert np.all(np.logical_not(temp[depth_name]<0)), 'depths should all be positive'
    assert np.all(np.logical_not(temp[depth_name].diff(depth_name)<=0)), 'depths should be monotonically increasing'
    
    temp = temp.load() # Unfortunately, where doesn't work with lazy data

    # Manage any NaNs coming in
    # In this case and the next, "manage" means fill with dummy values I know will work, 
    # and then replace them all later
    originally_nan = np.isnan(temp.isel({depth_name:0})) #Probably land - any lat/lon where the surface is a NaN
    temp = temp.where(np.logical_not(originally_nan)| (temp[depth_name]>temp[depth_name][0]),target_temp+0.1) # If NaN, reset top to just above isotherm
    temp = temp.where(np.logical_not(originally_nan) | (temp[depth_name]!=temp[depth_name][1]),target_temp-0.1) #If NaN, 2nd level to just below isotherm

    # Manage any locations which don't have an appropriate isotherm (never above target_temp)
    isotherm_outcrops = (temp.max(depth_name)<target_temp)
    temp = temp.where(np.logical_not(isotherm_outcrops) | (temp[depth_name]>temp[depth_name][0]),target_temp+0.1) # If it outcrops, reset top to 20.1

    # Find deepest point above target temperature
    i0 = (temp[depth_name].where(temp>=target_temp)          # Depth where warmer than target
         ).argmax(dim=depth_name).compute()                 # Find deepest index

    assert np.all(i0<(temp[depth_name].shape[0]-1)), 'bottom cell is above target_temp, not designed to handle this condition'

    # Depth immediately below isotherm
    i1 = i0+1

    t0 = temp.isel({depth_name:i0})             # temperature above isotherm
    z0 = temp[depth_name].isel({depth_name:i0}) # depth above isotherm

    t1 = temp.isel({depth_name:i1})             # temperature below isotherm
    z1 = temp[depth_name].isel({depth_name:i1}) # depth below isotherm

    # Manage any locations where z0 is just on top of bathymetry
    isotherm_hits_bathymetry = np.isnan(t1)

    isotherm_depth = (target_temp-t0)*(z0-z1)/(t0-t1)+z0 # Linear interpolation, from rearranging (y-y0) = m(x-x0)

    # Add back in all of our caveats (in order of increasing priority)
    isotherm_depth = isotherm_depth.where(np.logical_not(isotherm_hits_bathymetry),(z0+z1)/2)
    isotherm_depth = isotherm_depth.where(np.logical_not(isotherm_outcrops),0)
    isotherm_depth = isotherm_depth.where(np.logical_not(originally_nan))
    
    
    return isotherm_depth.drop_vars(depth_name)

### Load and plot data from ACCESS-OM3

In [5]:
exptname=os.path.basename(os.path.dirname(esm_file))

datastore = intake.open_esm_datastore(
    esm_file,
    columns_with_iterables=[
        "variable",
        "variable_long_name",
        "variable_standard_name",
        "variable_cell_methods",
        "variable_units"
    ]
)

In [6]:
exptname

'MC_25km_jra_ryf-1.0-beta'

In [7]:
geolon = datastore.search(variable="geolon").to_dask().geolon
geolat = datastore.search(variable="geolat").to_dask().geolat

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.08/lib/python3.11/site-packages/intake_esm/source.py:306: ConcatenationWarning: Attempting to concatenate datasets without valid dimension coordinates: retaining only first dataset. Request valid dimension coordinate to silence this warning.
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.08/lib/python3.11/site-packages/intake_esm/source.py:306: ConcatenationWarning: Attempting to concatenate datasets without valid dimension coordinates: retaining only first dataset. Request valid dimension coordinate to silence this warning.
  warnings.warn(


In [11]:
variable = "thetao"
model_all = datastore.search(variable=variable, frequency="1mon").to_dask(
    xarray_open_kwargs = dict(
        chunks={"time": -1},
        decode_timedelta=True
    ),
    xarray_combine_by_coords_kwargs=dict(
        compat="override",
        data_vars="minimal",
        coords="minimal"
    )
)[variable].cf.assign_coords({ "longitude": geolon, "latitude": geolat })

ValueError: Expected exactly one dataset. Received 0 datasets. Please refine your search or use `.to_dataset_dict()`.

In [16]:
sorted(datastore.unique()['variable'])

['ANGLE',
 'ANGLET',
 'Coriolis',
 'ELAT',
 'ELON',
 'HTE',
 'HTN',
 'KE',
 'NCAT',
 'NFSD',
 'NLAT',
 'NLON',
 'SSH_max',
 'SSH_min',
 'TLAT',
 'TLON',
 'Tair_m',
 'Tsfc_m',
 'ULAT',
 'ULON',
 'VGRDa',
 'VGRDb',
 'VGRDi',
 'VGRDs',
 'agessc',
 'aice',
 'aice_m',
 'aicen_m',
 'albice_m',
 'albsni_m',
 'albsno_m',
 'alidf_ai_m',
 'alidr_ai_m',
 'alvdf_ai_m',
 'alvdr_ai_m',
 'areacello',
 'areacello_bu',
 'areacello_cu',
 'areacello_cv',
 'average_DT',
 'average_T1',
 'average_T2',
 'blkmask',
 'congel',
 'congel_m',
 'cos_rot',
 'daidtd_m',
 'daidtt_m',
 'deptho',
 'divu_m',
 'dvidtd',
 'dvidtd_m',
 'dvidtt',
 'dvidtt_m',
 'dxCu',
 'dxCv',
 'dxCvo',
 'dxe',
 'dxn',
 'dxt',
 'dxu',
 'dyCu',
 'dyCuo',
 'dyCv',
 'dye',
 'dyn',
 'dyt',
 'dyu',
 'e',
 'earea',
 'emask',
 'evap_ai_m',
 'fcondtop_ai_m',
 'fcondtopn_ai_m',
 'fhocn_ai_m',
 'flat_ai_m',
 'flatn_ai_m',
 'flwdn_m',
 'flwup_ai_m',
 'fmeltt_ai_m',
 'fmelttn_ai_m',
 'frazil',
 'frazil_m',
 'fresh_ai_m',
 'frzmlt',
 'frzmlt_m',
 'fsalt